In [1]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import numpy as np
import ventTools
from ventTools.data import convert_to_fixed_length
from ast import literal_eval
import ventmode

In [2]:
# defining features of record
@tf.function 
def map_fn(serialized_example):
    feature = {
        'label'                 : tf.io.FixedLenFeature(1, tf.int64),
        'flow'                  : tf.io.FixedLenFeature(202, tf.float32),
        'pressure'              : tf.io.FixedLenFeature(202, tf.float32),
        'y'                     : tf.io.FixedLenFeature(1, tf.float32),
        #'volume'                : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_diff'             : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_diff'         : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_1lag'             : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_2lag'             : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_3lag'             : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_1lag'         : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_2lag'         : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_3lag'         : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_1oracle'          : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_2oracle'          : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_3oracle'          : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_1oracle'      : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_2oracle'      : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_3oracle'      : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_cumsum'           : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_cumsum'       : tf.io.FixedLenFeature(200, tf.float32),        
    }
    return tf.io.parse_single_example(serialized_example, feature)

@tf.function 
def map_all(serialized):
    ex = map_fn(serialized)
    return (tf.stack([ex["flow"], ex["pressure"]],1), ex["y"]) 
                        #ex['volume'], 
                        #ex['flow_diff'], ex['pressure_diff'], 
                        #ex['flow_1lag'], ex['flow_2lag'], ex['flow_3lag'], 
                        #ex['pressure_1lag'], ex['pressure_2lag'], ex['pressure_3lag'], 
                        #ex['flow_1oracle'], ex['flow_2oracle'], ex['flow_3oracle'], 
                        #ex['pressure_1oracle'], ex['pressure_2oracle'], ex['pressure_3oracle'], ex['flow_cumsum'], ex['pressure_cumsum']], 1), ex["y"])
  

@tf.function
def get_dist(targets, n_classes, window_size):
  shape = (n_classes,)
  res = tf.zeros(shape)

  for i in range(window_size):
    idx = tf.gather(targets, i, axis=0)
    mat = tf.sparse.SparseTensor([[idx]], [1.], shape)
    res = tf.sparse.add(res, mat)
    
  res = tf.math.divide(res, window_size)
  return tf.ensure_shape(res, shape)

@tf.function
def map_window(ds, window_size, input_shape):
  feat = tf.zeros([0] + list(input_shape))
  y = tf.zeros([0, ])
  
  for sample in ds:
    ex = map_all(sample)

    features = ex[0]
    features = tf.convert_to_tensor(features)
    features = tf.expand_dims(features, axis=0)

    feat = tf.concat([feat, features], axis=0)
    y = tf.concat([y, ex[1]], axis=0)
    
  feat = tf.ensure_shape(feat, (window_size, 200, 17))
  y = tf.ensure_shape(y, (window_size,))
  return (feat, y)


window_size = 10
x_shape = (200, 19)
n_classes = 5
calc_dist = lambda x, y: (x, get_dist(y, n_classes, window_size))

train_set = tf.data.TFRecordDataset('train_raw.tfrecord')
train_set = train_set.map(map_all)


test_dataset = tf.data.TFRecordDataset('test_raw.tfrecord')
test_dataset = test_dataset.map(map_all)

train_size = sum(1 for _ in train_set)

transpose_set = train_set.map(lambda x,y : (tf.transpose(x),y))

transpose_test_set = test_dataset.map(lambda x,y : (tf.transpose(x),y))


In [3]:
def encode(file_no_ext, data_set, fileName):
    json_file = open(file_no_ext + ".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = keras.models.model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(file_no_ext + ".h5")
    
    # removemlp
    model = keras.Sequential([
        loaded_model.layers[0]
    ])
    model.build(input_shape = (None,2,202))
    
    # get prediction
    features = [y for y in model.predict(data_set.batch(128))] 
    result = pd.DataFrame(features)
    y_val = [y.numpy()[0] for _,y in data_set]
    y_series = pd.Series(y_val)
    
    from pathlib import Path
    file = Path(fileName)
    result["y"] = y_series

    # save results to csv or another format

    result.to_csv(file)
    return result

In [21]:
# get features from transformer
def encodeT(file_no_ext, data_set, fileName):
    json_file = open(file_no_ext + ".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = keras.models.model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(file_no_ext + ".h5")
    print(loaded_model.layers[12])

    model = keras.Model(loaded_model.layers[0].input, loaded_model.layers[12].output)
    
    # get prediction
    features = [y for y in model.predict(data_set.batch(128))] 
    result = pd.DataFrame(features)
    y_val = [y.numpy()[0] for _,y in data_set]
    y_series = pd.Series(y_val)
    
    # save results to csv or another format
    from pathlib import Path
    file = Path(fileName)
    result["y"] = y_series

    result.to_csv(file)
    return result 

In [ ]:
encodeT("transformer_cross2", train_set, "transformer_train_cross2")
encodeT("transformer_cross2", test_dataset, "transformer_test_cross2")

In [ ]:
encode("lstm_model_cross2", transpose_set, "lstm_train_cross2")
encode("lstm_model_cross2", transpose_test_set, "lstm_test_cross2")

encode("conv_model_cross2",transpose_set, "cnn_train_cross2")
encode("conv_model_cross2",transpose_test_set, "cnn_test_cross2")

encode("ff_model_cross2", transpose_set, "ff_train_cross2")
encode("ff_model_cross2", transpose_test_set, "ff_test_cross2")